In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [15]:
import sys
sys.path.append('/content/gdrive/MyDrive/TFG/notebooks/')

In [16]:
from functions import *

In [209]:
import matplotlib.pyplot as plt
import glob
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import geopandas as gpd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

## Load data

In [104]:
lst = '/content/gdrive/MyDrive/TFG/data/preprocessed/lst'
wt = '/content/gdrive/MyDrive/TFG/data/preprocessed/wt'
wt_interpolated = '/content/gdrive/MyDrive/TFG/data/preprocessed/wt_interpolated'
masked = '/content/gdrive/MyDrive/TFG/data/preprocessed/lst/masked'
ndvi = '/content/gdrive/MyDrive/TFG/data/preprocessed/ndvi'

In [189]:
lst_df = pd.read_csv(f'{lst}/leboiron_lst.csv',index_col=0)
targets_df = pd.read_csv(f'{wt}/leboiron_wt.csv',index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/TFG/data/preprocessed/wt/leboiron_wt.csv'

In [192]:
targets_df

,29,30,31,32,285,286,287,288,541,542,...,65207,65208,65209,65210,65461,65462,65463,65464,65465,65466
2013-06,2.870015,9.654258,7.900476,3.271294,3.759435,9.420562,4.067366,0.000080,0.718058,9.883817,...,6.145782,1.171358,0.000238,0.000225,0.000033,0.000169,1.545363,7.782999,9.784155,9.248206
2014-01,1.629292,5.479551,4.483296,1.856194,2.134468,5.350365,2.310644,0.000045,0.408157,5.617906,...,3.027052,0.577923,0.000118,0.000112,0.000016,0.000083,0.762449,3.846584,4.845422,4.584611
2014-09,3.470196,11.672450,9.551773,3.954965,4.545773,11.391972,4.918897,0.000096,0.868501,11.954502,...,6.936604,1.322857,0.000270,0.000255,0.000037,0.000190,1.745233,8.795513,11.066760,10.467578
2015-03,1.875099,6.305809,5.159029,2.135947,2.456527,6.157527,2.659155,0.000052,0.469722,6.465285,...,3.505339,0.668805,0.000136,0.000129,0.000019,0.000096,0.882348,4.448438,5.599468,5.296111
2016-07,3.718413,12.508859,10.237086,4.238910,4.870588,12.203864,5.268697,0.000103,0.929993,12.801191,...,7.862251,1.499030,0.000305,0.000289,0.000042,0.000215,1.977657,9.964449,12.533132,11.851870
2016-09,3.704787,12.462427,10.198731,4.222960,4.852862,12.160157,5.250077,0.000103,0.926800,12.757146,...,7.566386,1.442626,0.000294,0.000278,0.000040,0.000207,1.903243,9.589507,12.060917,11.404564
2016-10,2.723724,9.160596,7.495629,3.103464,3.568168,8.943588,3.862276,0.000076,0.682141,9.389137,...,5.089096,0.970647,0.000198,0.000187,0.000027,0.000139,1.280566,6.454267,8.120408,7.679015
2016-12,1.547124,5.201037,4.254041,1.761032,2.027250,5.083947,2.196395,0.000043,0.388300,5.344253,...,2.237978,0.427301,0.000087,0.000083,0.000012,0.000061,0.563734,2.843848,3.581138,3.386405
2017-01,0.850973,2.858668,2.336710,0.967038,1.115509,2.800255,1.210750,0.000024,0.214427,2.950813,...,0.744969,0.142829,0.000029,0.000028,0.000004,0.000020,0.188433,0.954024,1.206234,1.141472
2017-03,2.076275,6.983463,5.714106,2.365906,2.719829,6.815886,2.942880,0.000058,0.519632,7.152456,...,4.138843,0.789359,0.000161,0.000152,0.000022,0.000113,1.041394,5.248202,6.602268,6.242376


In [242]:
use_ndvi = False
if use_ndvi:
  ndvi_df = pd.read_csv(f'{ndvi}/leboiron_ndvi.csv',index_col=0)
  df_combined = pd.concat([lst_df, ndvi_df], axis=1)
  train_input, train_target, validation_input, validation_target, test_input, test_target = split_data_df(df_combined, targets_df)
  X = df_combined

else:
  train_input, train_target, validation_input, validation_target, test_input, test_target = split_data_df(lst_df, targets_df)
  X = lst_df

y = targets_df

## Linear Regression model

In [106]:
# Initialize the model
model = LinearRegression()

# Train the model using the training data
model.fit(train_input, train_target)

LinearRegression()

### Validate and test

In [107]:
validation_prediction = model.predict(validation_input)

In [108]:
results = evaluate_model(validation_target, validation_prediction)
for k,v in results.items():
  print(k,':',v)

MAE : 4.1274129136437105
MSE : 288.4323407005618
RMSE : 9.8232620329521
R² : -783.1131162972489
MAPE (%) : 259.8371828096996
MSE sample-wise : 96.49647696803822


In [109]:
test_prediction = model.predict(test_input)

In [110]:
results = evaluate_model(test_target, test_prediction)
for k,v in results.items():
  print(k,':',v)

MAE : 1.1361065885140544
MSE : 1.4312126822910647
RMSE : 1.7471855013439959
R² : -5.769177253348616
MAPE (%) : 46.94215343229184
MSE sample-wise : 3.0526571761066705


Image-wise error

In [111]:
c = 0
mean_results = {k:[] for k in results.keys()}
# Loop through each sample and compute the MSE for that sample
for i in range(test_target.shape[0]):
    res = evaluate_model(test_target[i], test_prediction[i])

    for k,v in res.items():
      mean_results[k].append(v)
    print('before': )

for key in mean_results:
    mean_results[key] = np.mean(mean_results[key])

print(mean_results)

{'MAE': 1.1361065885140542, 'MSE': 10.208489821590659, 'RMSE': 1.7083935889459099, 'R²': 0.5181263738207087, 'MAPE (%)': 46.94215343229184, 'MSE sample-wise': 3.0526571761066705}


### K-fold cross-validation

In [234]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [235]:
df_scaled = pd.DataFrame(X_scaled, columns=X.columns)

In [237]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

model = LinearRegression()

In [238]:
mean_results_kfold = {k:[] for k in results.keys()}  # Initialize empty lists for each metric

# Iterate over the folds
for train_index, test_index in kf.split(df_scaled):
    X_train, X_test = df_scaled.iloc[train_index], df_scaled.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Initialize mean_results for this fold (to store per-image metrics)
    mean_results = {k:[] for k in results.keys()}

    # Iterate over each image in the test set
    for i in range(y_test.shape[0]):  # For each image (row) in the test set
        # Calculate the metrics for the image
        results = evaluate_model(y_test.iloc[i], y_pred[i])

        # Store the results for this image
        for k, v in results.items():
            mean_results[k].append(v)

    # Compute the average metric for this fold and store in mean_results_kfold
    for key in mean_results:
        fold_mean = np.mean(mean_results[key])
        mean_results_kfold[key].append(fold_mean)  # Append fold mean to global results


Streaming output truncated to the last 5000 lines.
/content/gdrive/MyDrive/TFG/notebooks/functions.py:156: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_true_flatten = y_true[i].flatten()
/content/gdrive/MyDrive/TFG/notebooks/functions.py:156: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_true_flatten = y_true[i].flatten()
/content/gdrive/MyDrive/TFG/notebooks/functions.py:156: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_tru

In [239]:
for key in mean_results_kfold:
    print('Mitja de', key,'amb ', mean_results_kfold[key])

final_mean_results = {k: np.mean(v) for k, v in mean_results_kfold.items()}


Mitja de MAE amb  [0.8612343182104351, 0.7398277904991541, 0.9051862189083529, 0.895350218677993, 1.8012057555778933]
Mitja de MSE amb  [8.398667425901058, 4.568961193558539, 6.646677108418028, 7.580710163148032, 26.48985264696239]
Mitja de RMSE amb  [1.3085373208367461, 1.1102420370148605, 1.3726944183114747, 1.340271140161635, 2.6904679049631053]
Mitja de R² amb  [-0.6719216788108401, 0.886939443652775, 0.8588310660372435, 0.812971846690092, -0.5566526233571706]
Mitja de MAPE (%) amb  [76.27709869209333, 21.440777614430434, 25.226074670516628, 27.35165484606005, 79.85624985387128]
Mitja de MSE sample-wise amb  [2.4003138959702404, 1.45158427078861, 2.312551932808536, 2.335678798308707, 8.10536525947038]


In [240]:
final_mean_results

{'MAE': 1.0405608603747658,
 'MSE': 10.73697370759761,
 'RMSE': 1.5644425642575643,
 'R²': 0.26603361084241994,
 'MAPE (%)': 46.03037113539434,
 'MSE sample-wise': 3.321098831469295}

### Save model results

In [241]:
model_name = "LR_5"
details = {'Input': 'RGB filtered lst+ndvi scaled', 'Output': 'Water stations', 'Resolution': 256, 'Batch size':'', 'Epochs':''}

file_path = "/content/gdrive/MyDrive/TFG/results/model_results_img_wise.xlsx"
save_excel(file_path, model_name, final_mean_results, excel = 'Results')

file_path = "/content/gdrive/MyDrive/TFG/results/model_details.xlsx"
save_excel(file_path, model_name, details, excel = 'Details')


## Physics regression

In [243]:
T_a_squared_df = lst_df ** 2
T_a_squared_df.columns = [col + '^2' for col in lst_df.columns]

In [244]:
X = pd.concat([lst_df, T_a_squared_df],axis=1)

Scale data

In [210]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [213]:
df_scaled = pd.DataFrame(X_scaled, columns=df_combined.columns)

### Split data and train test model

In [214]:
train_input, train_target, validation_input, validation_target, test_input, test_target = split_data_df(df_scaled, targets_df)

In [215]:
# Initialize the model
model = LinearRegression()

# Train the model using the training data
model.fit(train_input, train_target)

LinearRegression()

In [216]:
validation_prediction = model.predict(validation_input)

In [217]:
results = evaluate_model(validation_target, validation_prediction)
for k,v in results.items():
  print(k,':',v)

MAE : 1.675405803812477
MSE : 8.508514486988604
RMSE : 2.564793738676436
R² : -0.3608143324054614
MAPE (%) : 46.51545151564784
MSE sample-wise : 6.5781669219538506


### K-fold cross-validation

In [245]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

model = LinearRegression()

In [ ]:
mean_results_kfold = {k:[] for k in results.keys()}  # Initialize empty lists for each metric

# Iterate over the folds
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Initialize mean_results for this fold (to store per-image metrics)
    mean_results = {k:[] for k in results.keys()}

    # Iterate over each image in the test set
    for i in range(y_test.shape[0]):  # For each image (row) in the test set
        # Calculate the metrics for the image
        results = evaluate_model(y_test.iloc[i], y_pred[i])

        # Store the results for this image
        for k, v in results.items():
            mean_results[k].append(v)

    # Compute the average metric for this fold and store in mean_results_kfold
    for key in mean_results:
        fold_mean = np.mean(mean_results[key])
        mean_results_kfold[key].append(fold_mean)  # Append fold mean to global results


Streaming output truncated to the last 5000 lines.
/content/gdrive/MyDrive/TFG/notebooks/functions.py:156: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_true_flatten = y_true[i].flatten()
/content/gdrive/MyDrive/TFG/notebooks/functions.py:156: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_true_flatten = y_true[i].flatten()
/content/gdrive/MyDrive/TFG/notebooks/functions.py:156: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_tru

In [248]:
for key in mean_results_kfold:
    print('Mitja de', key,'amb ', mean_results_kfold[key])

final_mean_results = {k: np.mean(v) for k, v in mean_results_kfold.items()}
final_mean_results

Mitja de MAE amb  [13.607991196356908, 6.43292693695243, 171.69906894834412, 5.157575596243587, 46.62744908395774]
Mitja de MSE amb  [5349.534443668887, 591.1517189668024, 1248617.9104077455, 771.3704381805035, 69229.75771047996]
Mitja de RMSE amb  [20.18882268127341, 9.566487077821154, 254.64846169320418, 7.864618159817377, 69.184247523581]
Mitja de R² amb  [-2214.794923109918, -7.042911376304664, -43936.684807034544, -32.24217145677094, -7345.9269697434875]
Mitja de MAPE (%) amb  [2106.620990891825, 153.74982728533436, 5856.622648600487, 200.76180297047858, 2793.619366476714]
Mitja de MSE sample-wise amb  [1885.5444965988015, 201.77008036060016, 428572.32633132255, 211.22749598401174, 22995.404587170855]


{'MAE': 48.705002352370954,
 'MSE': 264911.9449438083,
 'RMSE': 72.29052742713944,
 'R²': -10707.338356544204,
 'MAPE (%)': 2222.2749272449682,
 'MSE sample-wise': 90773.25459828737}

### Save model results

In [249]:
model_name = "Physics-based LR_2"
details = {'Input': 'RGB filtered lst', 'Output': 'Water stations', 'Resolution': 256, 'Batch size':'', 'Epochs':''}

file_path = "/content/gdrive/MyDrive/TFG/results/model_results_img_wise.xlsx"
save_excel(file_path, model_name, final_mean_results, excel = 'Results')

file_path = "/content/gdrive/MyDrive/TFG/results/model_details.xlsx"
save_excel(file_path, model_name, details, excel = 'Details')
